In [1]:
import requests 
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup
from tqdm import tqdm
import multiprocessing

In [2]:
df = pd.read_csv('celeb(1).csv')
df.head()

,celeb_id,celeb_name,profession,famous_for,img_link,profile_link
0,nm0000079,Raquel Welch,Actress,The Three Musketeers,https://m.media-amazon.com/images/M/MV5BZDJhMz...,https://www.imdb.com/name/nm0000079
1,nm0050959,Pedro Pascal,Actor,The Mandalorian,https://m.media-amazon.com/images/M/MV5BMGNlYT...,https://www.imdb.com/name/nm0050959
2,nm8165602,Bella Ramsey,Actress,Game of Thrones,https://m.media-amazon.com/images/M/MV5BMWJkZW...,https://www.imdb.com/name/nm8165602
3,nm0001491,Melanie Lynskey,Actress,Togetherness,https://m.media-amazon.com/images/M/MV5BMjIzND...,https://www.imdb.com/name/nm0001491
4,nm8567338,Sasha Calle,Actress,The Flash,https://m.media-amazon.com/images/M/MV5BZTc1Ym...,https://www.imdb.com/name/nm8567338


In [3]:
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36 Edg/103.0.1264.49"}

def fetcher(df,start,end):
    data = []
    for i in tqdm(df['profile_link'][start:end]):
        webdata = requests.get(i,headers = headers).text
        soup = BeautifulSoup(webdata,'lxml')

        # professsion of the actor or the actress
        try:
            prof = ','.join([prof.text.strip() for prof in soup.find('ul',class_ = 'ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt').find_all('li')])
        except:
            prof = np.nan

        # films he/she is known for 
        try:
            famous = ','.join([fame.text for fame in soup.find_all('div',class_ = 'ipc-primary-image-list-card__content-top')])
        except:
            famous = np.nan

        try:
            for i in soup.find('div',class_ = 'sc-f65f65be-0 fVkLRr').find_all('li'):
                if 'height'.lower() in i.text.lower():                                   # height
                    height            = i.text[13:-1]

                if 'born'.lower() in i.text.lower():                                     # birth date and birth place
                    dob               = i.find('div').find_all('li')[0].text
                    birthplace        = i.find('div').find_all('li')[1].text
        except:
            height = np.nan
            dob = np.nan 
            birthplace = np.nan

        data.append([prof,awards, famous, height, dob, birthplace])         
    df_ = pd.DataFrame(data,columns = ['prof','awards', 'famous', 'height', 'dob', 'birthplace'])
    df_.to_csv('checkpoints/'+ str(end) +'.csv',index = False )

In [4]:
# multiprocessing 

In [5]:
p1 = multiprocessing.Process(target = fetcher,args = (df,0,1001))
p2 = multiprocessing.Process(target = fetcher,args = (df,1001,2001))
p3 = multiprocessing.Process(target = fetcher,args = (df,2001,3001))
p4 = multiprocessing.Process(target = fetcher,args = (df,3001,4001))
p5 = multiprocessing.Process(target = fetcher,args = (df,4001,5000))

p1.start()
p2.start()
p3.start()
p4.start()
p5.start()

p1.join()
p2.join()
p3.join()
p4.join()
p5.join()


100%|█████████████████████████████████████| 1001/1001 [1:04:59<00:00,  3.90s/it]


In [ ]:
# combining of the dataset 

In [6]:
### reading all the csv files in the checkpoints
df_1 = pd.read_csv('checkpoints/1001.csv')
df_2 = pd.read_csv('checkpoints/2001.csv')
df_3 = pd.read_csv('checkpoints/3001.csv')
df_4 = pd.read_csv('checkpoints/4001.csv')
df_5 = pd.read_csv('checkpoints/5001.csv')    

In [18]:
### concating the dataset into a single dataframe
df_ = pd.concat([df_1,df_2,df_3,df_4,df_5])

In [37]:
df_

,prof,famous,height,dob,birthplace
0,"Actress,Producer,Writer","The Three Musketeers,Myra Breckinridge,Hannie ...",1.68 m,"September 5, 1940","Chicago, Illinois, USA"
1,"Actor,Producer,Soundtrack","The Mandalorian,Kingsman: The Golden Circle,Ga...",(1.79 m,"April 2, 1975","Santiago de Chile, Metropolitan Region, Chile"
2,"Actress,Soundtrack,Writer","Game of Thrones,The Last of Us,Catherine Calle...",(1.56 m,"September 30, 2003","Nottinghamshire, England, UK"
3,"Actress,Soundtrack","Togetherness,Heavenly Creatures,Up in the Air,...",1.70 m,"May 16, 1977","New Plymouth, New Zealand"
4,"Actress,Music Department","The Flash,Supergirl,On Swift Horses,The Young ...",1.70 m,"August 7, 1995","Boston, Massachusetts, USA"
...,...,...,...,...,...
993,"Actress,Producer","The Accountant,The Americans,The Nanny Diaries...",1.65 m,"July 12, 1976","Sunderland, Tyne & Wear, England, UK"
994,"Actor,Soundtrack","Monk,A Beautiful Mind,Flags of Our Fathers,The...",(1.80 m,"May 19, 1970","Vancouver, British Columbia, Canada"
995,"Actress,Soundtrack","Spies Like Us,Wayne's World,Twilight Zone: The...",1.75 m,"July 20, 1957","Alexandria, Virginia, USA"
996,"Actor,Writer,Producer","Deadwood,Supernatural,Breaking Bad,The Boys",(1.80 m,"August 12, 1950","Laramie, Wyoming, USA"


In [47]:
# combining the dataset with the celeb (1).csv dataset 


In [81]:
id_           = []
names         = []
profession    = []
famous_for    = []
img_link      = []
profile_link  = []
worked_as     = []
famous_movies = []
height        = []
dob           = []
birthplace    = []


for i in df.values:
    id_.append(i[0]) 
    names.append(i[1])
    profession.append(i[2])
    famous_for.append(i[3])
    img_link.append(i[4])
    profile_link.append(i[5])

for i in df_.values:
    worked_as.append(i[0])
    famous_movies.append(i[1])
    height.append(i[2])
    dob.append(i[3])
    birthplace.append(i[4])

    
final = pd.DataFrame()
final['id_']           = id_
final['names']         = names        
final['profession']    = profession   
final['worked_as']     = worked_as         
final['famous_for']    = famous_for   
final['famous_movies'] = famous_movies
final['height']        = height       
final['dob']           = dob          
final['birthplace']    = birthplace   
final['img_link']      = img_link     
final['profile_link']  = profile_link 

final.head()

,id_,names,profession,worked_as,famous_for,famous_movies,height,dob,birthplace,img_link,profile_link
0,nm0000079,Raquel Welch,Actress,"Actress,Producer,Writer",The Three Musketeers,"The Three Musketeers,Myra Breckinridge,Hannie ...",1.68 m,"September 5, 1940","Chicago, Illinois, USA",https://m.media-amazon.com/images/M/MV5BZDJhMz...,https://www.imdb.com/name/nm0000079
1,nm0050959,Pedro Pascal,Actor,"Actor,Producer,Soundtrack",The Mandalorian,"The Mandalorian,Kingsman: The Golden Circle,Ga...",(1.79 m,"April 2, 1975","Santiago de Chile, Metropolitan Region, Chile",https://m.media-amazon.com/images/M/MV5BMGNlYT...,https://www.imdb.com/name/nm0050959
2,nm8165602,Bella Ramsey,Actress,"Actress,Soundtrack,Writer",Game of Thrones,"Game of Thrones,The Last of Us,Catherine Calle...",(1.56 m,"September 30, 2003","Nottinghamshire, England, UK",https://m.media-amazon.com/images/M/MV5BMWJkZW...,https://www.imdb.com/name/nm8165602
3,nm0001491,Melanie Lynskey,Actress,"Actress,Soundtrack",Togetherness,"Togetherness,Heavenly Creatures,Up in the Air,...",1.70 m,"May 16, 1977","New Plymouth, New Zealand",https://m.media-amazon.com/images/M/MV5BMjIzND...,https://www.imdb.com/name/nm0001491
4,nm8567338,Sasha Calle,Actress,"Actress,Music Department",The Flash,"The Flash,Supergirl,On Swift Horses,The Young ...",1.70 m,"August 7, 1995","Boston, Massachusetts, USA",https://m.media-amazon.com/images/M/MV5BZTc1Ym...,https://www.imdb.com/name/nm8567338


In [82]:
# finally saving the file of scraped data
final.to_csv('Celebrity_scraper.csv',index = False)

In [196]:
df = pd.read_csv('Celebrity_scraper.csv')
df.head()

,id_,names,profession,worked_as,famous_for,famous_movies,height,dob,birthplace,img_link,profile_link
0,nm0000079,Raquel Welch,Actress,"Actress,Producer,Writer",The Three Musketeers,"The Three Musketeers,Myra Breckinridge,Hannie ...",1.68 m,"September 5, 1940","Chicago, Illinois, USA",https://m.media-amazon.com/images/M/MV5BZDJhMz...,https://www.imdb.com/name/nm0000079
1,nm0050959,Pedro Pascal,Actor,"Actor,Producer,Soundtrack",The Mandalorian,"The Mandalorian,Kingsman: The Golden Circle,Ga...",(1.79 m,"April 2, 1975","Santiago de Chile, Metropolitan Region, Chile",https://m.media-amazon.com/images/M/MV5BMGNlYT...,https://www.imdb.com/name/nm0050959
2,nm8165602,Bella Ramsey,Actress,"Actress,Soundtrack,Writer",Game of Thrones,"Game of Thrones,The Last of Us,Catherine Calle...",(1.56 m,"September 30, 2003","Nottinghamshire, England, UK",https://m.media-amazon.com/images/M/MV5BMWJkZW...,https://www.imdb.com/name/nm8165602
3,nm0001491,Melanie Lynskey,Actress,"Actress,Soundtrack",Togetherness,"Togetherness,Heavenly Creatures,Up in the Air,...",1.70 m,"May 16, 1977","New Plymouth, New Zealand",https://m.media-amazon.com/images/M/MV5BMjIzND...,https://www.imdb.com/name/nm0001491
4,nm8567338,Sasha Calle,Actress,"Actress,Music Department",The Flash,"The Flash,Supergirl,On Swift Horses,The Young ...",1.70 m,"August 7, 1995","Boston, Massachusetts, USA",https://m.media-amazon.com/images/M/MV5BZTc1Ym...,https://www.imdb.com/name/nm8567338


In [197]:
# checking the null values 
df.isnull().sum()

id_               0
names             0
profession        0
worked_as         3
famous_for        0
famous_movies     0
height           62
dob              64
birthplace       64
img_link          0
profile_link      0
dtype: int64

In [198]:
# Dealing with the null values 

In [199]:
df = df.dropna()

In [200]:
# exploring the dataset

In [201]:
df.describe()

,id_,names,profession,worked_as,famous_for,famous_movies,height,dob,birthplace,img_link,profile_link
count,4933,4933,4933,4933,4933,4933,4933,4933,4933,4933,4933
unique,4933,4932,13,321,2974,4931,94,4282,1947,4926,4933
top,nm0000079,Peyton List,Actress,Actress,Game of Thrones,Ginny & Georgia,1.70 m,November 24,"New York City, New York, USA",https://m.media-amazon.com/images/S/sash/9FayP...,https://www.imdb.com/name/nm0000079
freq,1,2,2551,517,28,2,388,5,279,8,1


In [202]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4933 entries, 0 to 4998
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_            4933 non-null   object
 1   names          4933 non-null   object
 2   profession     4933 non-null   object
 3   worked_as      4933 non-null   object
 4   famous_for     4933 non-null   object
 5   famous_movies  4933 non-null   object
 6   height         4933 non-null   object
 7   dob            4933 non-null   object
 8   birthplace     4933 non-null   object
 9   img_link       4933 non-null   object
 10  profile_link   4933 non-null   object
dtypes: object(11)
memory usage: 462.5+ KB


In [203]:
# pre -processing the heigth column 

In [204]:
height =[]
for i in df['height']:
    if i[-1] == 'm':
        if '(' == i[1] or '(' in i:
            height.append(float(i.strip()[1:-2]))
        else:
            height.append(float(i[:-2]))
    else:
        height.append(np.nan)

df['height'] = height

In [205]:
df.isnull().sum()

id_              0
names            0
profession       0
worked_as        0
famous_for       0
famous_movies    0
height           8
dob              0
birthplace       0
img_link         0
profile_link     0
dtype: int64

In [206]:
df = df.dropna()

In [207]:
# pre-processing the birthplace

In [208]:
country = []
state   = []
county  = []
city    = []
subcity = []
place   = []

for i in df['birthplace']:
    country.append(i.split(',')[-1].strip())
    if len(i.split(',')) == 6:
        state.append(i.split(',')[-2].strip())
        county.append(i.split(',')[-3].strip())
        city.append(i.split(',')[-4].strip())
        subcity.append(i.split(',')[1])
        place.append(i.split(',')[0].strip())
        
    elif len(i.split(',')) == 5 :
        state.append(i.split(',')[-2].strip())
        county.append('Not Available')
        city.append(i.split(',')[-3].strip())
        subcity.append(i.split(',')[1])
        place.append(i.split(',')[0].strip())
        
        
    elif len(i.split(',')) == 4:
        state.append(i.split(',')[-2].strip())
        city.append(i.split(',')[1].strip())
        county.append('Not Available')
        subcity.append('Not Available')
        place.append(i.split(',')[0].strip())
        
        
    elif len(i.split(',')) == 3:
        state.append(i.split(',')[-2].strip())
        city.append(i.split(',')[0].strip())
        county.append('Not Available')
        subcity.append('Not Available')
        place.append('Not Available')
        
    elif len(i.split(',')) == 2:
        state.append(i.split(',')[0])
        county.append('Not Available')
        city.append('Not available')
        subcity.append('Not Available')
        place.append('Not Available')
        
        
    else:
        state.append('Not Available')
        city.append('Not available')
        county.append('Not Available')
        subcity.append('Not Available')
        place.append('Not Available')

In [209]:
df['place  '] = place 
df['subcity'] = subcity
df['city   '] = city  
df['county '] = county 
df['state  '] = state  
df['country'] = country

df.head()

,id_,names,profession,worked_as,famous_for,famous_movies,height,dob,birthplace,img_link,profile_link,place,subcity,city,county,state,country
0,nm0000079,Raquel Welch,Actress,"Actress,Producer,Writer",The Three Musketeers,"The Three Musketeers,Myra Breckinridge,Hannie ...",1.68,"September 5, 1940","Chicago, Illinois, USA",https://m.media-amazon.com/images/M/MV5BZDJhMz...,https://www.imdb.com/name/nm0000079,Not Available,Not Available,Chicago,Not Available,Illinois,USA
1,nm0050959,Pedro Pascal,Actor,"Actor,Producer,Soundtrack",The Mandalorian,"The Mandalorian,Kingsman: The Golden Circle,Ga...",1.79,"April 2, 1975","Santiago de Chile, Metropolitan Region, Chile",https://m.media-amazon.com/images/M/MV5BMGNlYT...,https://www.imdb.com/name/nm0050959,Not Available,Not Available,Santiago de Chile,Not Available,Metropolitan Region,Chile
2,nm8165602,Bella Ramsey,Actress,"Actress,Soundtrack,Writer",Game of Thrones,"Game of Thrones,The Last of Us,Catherine Calle...",1.56,"September 30, 2003","Nottinghamshire, England, UK",https://m.media-amazon.com/images/M/MV5BMWJkZW...,https://www.imdb.com/name/nm8165602,Not Available,Not Available,Nottinghamshire,Not Available,England,UK
3,nm0001491,Melanie Lynskey,Actress,"Actress,Soundtrack",Togetherness,"Togetherness,Heavenly Creatures,Up in the Air,...",1.70,"May 16, 1977","New Plymouth, New Zealand",https://m.media-amazon.com/images/M/MV5BMjIzND...,https://www.imdb.com/name/nm0001491,Not Available,Not Available,Not available,Not Available,New Plymouth,New Zealand
4,nm8567338,Sasha Calle,Actress,"Actress,Music Department",The Flash,"The Flash,Supergirl,On Swift Horses,The Young ...",1.70,"August 7, 1995","Boston, Massachusetts, USA",https://m.media-amazon.com/images/M/MV5BZTc1Ym...,https://www.imdb.com/name/nm8567338,Not Available,Not Available,Boston,Not Available,Massachusetts,USA


In [210]:
df

,id_,names,profession,worked_as,famous_for,famous_movies,height,dob,birthplace,img_link,profile_link,place,subcity,city,county,state,country
0,nm0000079,Raquel Welch,Actress,"Actress,Producer,Writer",The Three Musketeers,"The Three Musketeers,Myra Breckinridge,Hannie ...",1.68,"September 5, 1940","Chicago, Illinois, USA",https://m.media-amazon.com/images/M/MV5BZDJhMz...,https://www.imdb.com/name/nm0000079,Not Available,Not Available,Chicago,Not Available,Illinois,USA
1,nm0050959,Pedro Pascal,Actor,"Actor,Producer,Soundtrack",The Mandalorian,"The Mandalorian,Kingsman: The Golden Circle,Ga...",1.79,"April 2, 1975","Santiago de Chile, Metropolitan Region, Chile",https://m.media-amazon.com/images/M/MV5BMGNlYT...,https://www.imdb.com/name/nm0050959,Not Available,Not Available,Santiago de Chile,Not Available,Metropolitan Region,Chile
2,nm8165602,Bella Ramsey,Actress,"Actress,Soundtrack,Writer",Game of Thrones,"Game of Thrones,The Last of Us,Catherine Calle...",1.56,"September 30, 2003","Nottinghamshire, England, UK",https://m.media-amazon.com/images/M/MV5BMWJkZW...,https://www.imdb.com/name/nm8165602,Not Available,Not Available,Nottinghamshire,Not Available,England,UK
3,nm0001491,Melanie Lynskey,Actress,"Actress,Soundtrack",Togetherness,"Togetherness,Heavenly Creatures,Up in the Air,...",1.70,"May 16, 1977","New Plymouth, New Zealand",https://m.media-amazon.com/images/M/MV5BMjIzND...,https://www.imdb.com/name/nm0001491,Not Available,Not Available,Not available,Not Available,New Plymouth,New Zealand
4,nm8567338,Sasha Calle,Actress,"Actress,Music Department",The Flash,"The Flash,Supergirl,On Swift Horses,The Young ...",1.70,"August 7, 1995","Boston, Massachusetts, USA",https://m.media-amazon.com/images/M/MV5BZTc1Ym...,https://www.imdb.com/name/nm8567338,Not Available,Not Available,Boston,Not Available,Massachusetts,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,nm2226071,Alison Wright,Actress,"Actress,Producer",The Accountant,"The Accountant,The Americans,The Nanny Diaries...",1.65,"July 12, 1976","Sunderland, Tyne & Wear, England, UK",https://m.media-amazon.com/images/M/MV5BMTZmNW...,https://www.imdb.com/name/nm2226071,Sunderland,Not Available,Tyne & Wear,Not Available,England,UK
4995,nm0337020,Jason Gray-Stanford,Actor,"Actor,Soundtrack",Monk,"Monk,A Beautiful Mind,Flags of Our Fathers,The...",1.80,"May 19, 1970","Vancouver, British Columbia, Canada",https://m.media-amazon.com/images/M/MV5BMjQ3MT...,https://www.imdb.com/name/nm0337020,Not Available,Not Available,Vancouver,Not Available,British Columbia,Canada
4996,nm0001146,Donna Dixon,Actress,"Actress,Soundtrack",Spies Like Us,"Spies Like Us,Wayne's World,Twilight Zone: The...",1.75,"July 20, 1957","Alexandria, Virginia, USA",https://m.media-amazon.com/images/M/MV5BMTkyMT...,https://www.imdb.com/name/nm0001146,Not Available,Not Available,Alexandria,Not Available,Virginia,USA
4997,nm0064769,Jim Beaver,Actor,"Actor,Writer,Producer",Deadwood,"Deadwood,Supernatural,Breaking Bad,The Boys",1.80,"August 12, 1950","Laramie, Wyoming, USA",https://m.media-amazon.com/images/M/MV5BMjExMj...,https://www.imdb.com/name/nm0064769,Not Available,Not Available,Laramie,Not Available,Wyoming,USA


In [211]:
del df['img_link']
del df['profile_link']
del df['profession']
del df['famous_for']

In [191]:
df

,id_,names,worked_as,famous_movies,height,dob,country,state,county,city,subcity,place
0,nm0000079,Raquel Welch,"Actress,Producer,Writer","The Three Musketeers,Myra Breckinridge,Hannie ...",1.68,"September 5, 1940",USA,Illinois,Not Available,Chicago,Not Available,Not Available
1,nm0050959,Pedro Pascal,"Actor,Producer,Soundtrack","The Mandalorian,Kingsman: The Golden Circle,Ga...",1.79,"April 2, 1975",Chile,Metropolitan Region,Not Available,Santiago de Chile,Not Available,Not Available
2,nm8165602,Bella Ramsey,"Actress,Soundtrack,Writer","Game of Thrones,The Last of Us,Catherine Calle...",1.56,"September 30, 2003",UK,England,Not Available,Nottinghamshire,Not Available,Not Available
3,nm0001491,Melanie Lynskey,"Actress,Soundtrack","Togetherness,Heavenly Creatures,Up in the Air,...",1.70,"May 16, 1977",New Zealand,New Plymouth,Not Available,Not available,Not Available,Not Available
4,nm8567338,Sasha Calle,"Actress,Music Department","The Flash,Supergirl,On Swift Horses,The Young ...",1.70,"August 7, 1995",USA,Massachusetts,Not Available,Boston,Not Available,Not Available
...,...,...,...,...,...,...,...,...,...,...,...,...
4994,nm2226071,Alison Wright,"Actress,Producer","The Accountant,The Americans,The Nanny Diaries...",1.65,"July 12, 1976",UK,England,Not Available,Tyne & Wear,Not Available,Sunderland
4995,nm0337020,Jason Gray-Stanford,"Actor,Soundtrack","Monk,A Beautiful Mind,Flags of Our Fathers,The...",1.80,"May 19, 1970",Canada,British Columbia,Not Available,Vancouver,Not Available,Not Available
4996,nm0001146,Donna Dixon,"Actress,Soundtrack","Spies Like Us,Wayne's World,Twilight Zone: The...",1.75,"July 20, 1957",USA,Virginia,Not Available,Alexandria,Not Available,Not Available
4997,nm0064769,Jim Beaver,"Actor,Writer,Producer","Deadwood,Supernatural,Breaking Bad,The Boys",1.80,"August 12, 1950",USA,Wyoming,Not Available,Laramie,Not Available,Not Available


In [212]:
# final save (Pre-processed data)


In [213]:
df.to_csv('celebs_preprocessed.csv',index = False)